# CZ3005 Project 1 - Balancing a Pole on a Cart

### Done by: Zon Liew (U1921098F), Charlotte Teo (U2022021G), Paul Low (U2022421F)

Objectives:
Apply Linear annealed policy with the EpsGreedyQPolicy as the inner policy:
Achieve a DQN model that trains in the least possible number of episodes.
Balance pole on the cart for 500 steps for 100 consecutive episodes while testing.

Epsilon-Greedy chooses the optimal action at each step, but sometimes randomly chooses an unlikely option.
We specify an initially high exploration rate (epsilon) of 1 at the beginning of Q function training because we know nothing about the importance of the Q table. Epsilon value is decreased as the agent has more confidence in the Q values.

A DQN agent can be used in any environment which has a discrete action space.
It is based on the Q - Network, a neural network model that can learn to predict Q-Values (expected returns) for all actions, given an observation from the environment.

The hyperparameters are:

Size of 1st fully connected layer: 256
Size of 2nd fully connected layer: 512
Period of the update of the target network parameters: 1000 steps
Discount factor: 0.99
Decay factor for epsilon in epsilon-greedy policy: 0.99
Minimum epsilon in epsilon-greeddy policy: 1E-4
Learning rate: 3E-4
Size of replay memory: 1000000
Period of experience replay: 4 steps
PER alpha: 0.2
PER beta0: 0.4



## Task 1


In [ ]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install gym[classic_control]
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1


# Import Dependencies

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
import matplotlib.pyplot as plt
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

In [ ]:
#Install keras rl2 which seamlessly integrates with the  OpenAI Gym  to evaluate and play around with DQN Algorithm
!pip install keras-rl2
!pip install dopamine-rl
#Install Open AI Gym for the Cart Pole Environment
!pip install tensorflow --upgrade
!pip install rl-agents==0.1.1

In [ ]:
import keras
from keras import Sequential
from keras.layers import Input, Flatten, Dense
import numpy as np
import rl
from rl.memory import SequentialMemory
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

from tensorflow.keras.optimizers import Adam

# Initialisation of Cartpole Environment
Loaded from Open AI gym suite

In [ ]:
#Load the CartPole environment from the OpenAI Gym suite
env = gym.make("CartPole-v1")

In [ ]:
#Resets the environment to an initial state and returns the initial observation.
initial_observation = env.reset()
print("Initial observation:", initial_observation)
cumulative_reward = 0
done = False


Observation corresponds to  'cart position', 'cart velocity', 'pole angle', and 'pole velocity' respectively.


# Initialisation of Agent

Setup of Deep Q-Network (DQN) agent using Keras-RL library for reinforcement learning.

Important functions used include:
1. SequentialMemory: This sets up an experience replay buffer with a capacity limit of 50,000 and a window length of 1. The buffer stores the agent's experiences so that they can be randomly sampled during the training process.
2. LinearAnnealedPolicy: This sets up the exploration policy for the agent. It uses the EpsGreedyQPolicy as the inner policy which selects actions based on a trade-off between exploration and exploitation. The LinearAnnealedPolicy gradually decreases the exploration rate (eps) from 1.0 to 0.1 over 10,000 steps.
3. Sequential: This sets up a feed-forward neural network model for the DQN. It has an input layer with a shape of (1, env.observation_space.shape[0]), which means it takes in a single observation vector of length env.observation_space.shape[0]. The hidden layers have 256 and 128 nodes respectively and use the ReLU activation function. The output layer has a number of nodes equal to the number of actions in the action space, and uses the linear activation function.
4. DQNAgent: This sets up the DQN agent using the previously defined model, memory, policy, and other hyperparameters such as the number of warmup steps and the target model update rate. The enable_dueling_network argument is set to True, which means the agent will use a dueling architecture to estimate the Q-values of each action.



In [ ]:
#Building DQN Agent with Keras-RL
# setup experience replay buffer
memory = SequentialMemory(limit=50000, window_length=1)

# setup the Linear annealed policy with the EpsGreedyQPolicy as the inner policy
policy =  LinearAnnealedPolicy(inner_policy=  EpsGreedyQPolicy(),   # policy used to select actions
                               attr='eps',                          # attribute in the inner policy to vary             
                               value_max=1.0,                       # maximum value of attribute that is varying
                               value_min=0.1,                       # minimum value of attribute that is varying
                               value_test=0.05,                     # test if the value selected is < 0.05
                               nb_steps=10000)                      # the number of steps between value_max and value_min

#Feed-Forward Neural Network Model for Deep Q Learning (DQN)
model = Sequential()
#print(env.observation_space)
#Input is 1 observation vector, and the number of observations in that vector 
model.add(Input(shape=(1,env.observation_space.shape[0])))
model.add(Flatten())
#Hidden layers with 24 nodes each
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
#Output is the number of actions in the action space
model.add(Dense(env.action_space.n, activation='linear')) 


#Feed-Forward Neural Network Architecture Summary
print(model.summary())

#Defining DQN Agent for DQN Model
dqn = DQNAgent(model=model,                     # Q-Network model
               nb_actions=env.action_space.n,   # number of actions
               memory=memory,                   # experience replay memory
               nb_steps_warmup=25,              # how many steps are waited before starting experience replay
               target_model_update=1e-2,        # how often the target network is updated
               policy=policy,                   # the action selection policy
              enable_dueling_network=True)                   

# Configure and compile agent. 
#Use built-in tensorflow.keras Adam optimizer and evaluation metrics            
#Adam._name = 'Adam'
dqn.compile(keras.optimizers.Adam(learning_rate=2.5e-4,epsilon = 0.01), metrics = ["mse",'accuracy'])



     

# Training 

In [ ]:
#Finally fit and train the agent
#Verbose parameter controls how much information is printed during training. A value of 10 means that training progress is printed every 10 steps.
history = dqn.fit(env, nb_steps=5000, visualize=False, verbose=10)


In [ ]:
# Visualize the history for number of Training episode steps of the Cart Pole Game
plt.figure(figsize = (18,10))
plt.plot(history.history['nb_episode_steps'])
plt.ylabel('nb_episode_steps')
plt.xlabel('episodes')
plt.show()


# Testing

In [ ]:
# Finally, evaluate and test our algorithm for 100 episodes.
dqn.test(env, nb_episodes=100, visualize=False)

In [ ]:
# After training is done, we save the final weights.
dqn.save_weights('dqn_weights.h5f', overwrite=False)

In [ ]:
observation = env.reset()
dqn.load_weights('dqn_weights.h5f')
action = dqn.forward(observation)

print("Observation: ", observation)
print("Chosen action: ", action)

new_observation, reward, done, info = env.step(action)
print("Observations after action: ", new_observation)
print("Reward for this step: ", reward)
print("Episode Completion: ", done)

## Task 2

Each Episode comprises of multiple steps. The environment is reset at the start of each episode as required. Next, the Agent makes the forward step based on the observation, before updating the new episode reward and observation for the next step. This continues until the termination criteria or truncation value of 500. Each cumulative reward is stored in a list and mapped out in the graph.

In [ ]:
num_episodes = 100
episode_results = []
dqn.load_weights('dqn_weights.h5f')
for i in range(1,num_episodes+1):
    # Reset environment at the beginning of each episode
    observation = env.reset()
    cumulative_reward = 0
    done = False

    while not done:
        # Agent takes action based on observation
        action = dqn.forward(observation)
        
        # Environment processes action and returns new observation, reward, and done flag
        new_observation, reward, done, info = env.step(action)

        # Update episode reward and observation for next step
        observation = new_observation
        cumulative_reward += reward
        
    print("Episode:", i, " Cumulative reward: ", cumulative_reward)
    episode_results.append(cumulative_reward)

In [ ]:
plt.plot(episode_results)
plt.title('Cumulative reward for each episode')
plt.ylabel('Cumulative reward')
plt.xlabel('episode')
plt.show()

In [ ]:
mean = sum(episode_results) / len(episode_results)
print("Average cumulative reward:", mean)
print("Is my agent good enough?", mean > 195)

## Task 3
Render an episode played by the developed RL agent

In [ ]:
env1 = RecordVideo(gym.make("CartPole-v1"), "./video")
observation = env1.reset()
while True:
    env1.render()
    #your agent goes here
    action = dqn.forward(observation)
    observation, reward, done, info = env.step(action) 
    if done: 
      break;    
env1.close()
show_video()